# Naive Bayes

##### Bayes’s Theorem

According to the Wikipedia, In probability theory and statistics,** Bayes’s theorem** (alternatively *Bayes’s law* or *Bayes’s rule*) describes the probability of an event, based on prior knowledge of conditions that might be related to the event.
Mathematically, it can be written as:

![formula.jpeg](attachment:formula.jpeg)

Where A and B are events and P(B)≠0
* P(A|B) is a conditional probability: the likelihood of event A occurring given that B is true.
* P(B|A) is also a conditional probability: the likelihood of event B occurring given that A is true.
* P(A) and P(B) are the probabilities of observing A and B respectively; they are known as the marginal probability.


Let’s understand it with the help of an example:

**The problem statement:**

You are planning a picnic today, but the morning is cloudy

Oh no! 50% of all rainy days start off cloudy!
But cloudy mornings are common (about 40% of days start cloudy)
And this is usually a dry month (only 3 of 30 days tend to be rainy, or 10%)
What is the chance of rain during the day?

We will use Rain to mean rain during the day, and Cloud to mean cloudy morning.

The chance of Rain given Cloud is written P(Rain|Cloud)

So let's put that in the formula:

$P(Rain|Cloud) = \frac{P(Rain)*P(Cloud|Rain)} {P(Cloud)}$          
                      


- P(Rain) is Probability of Rain = 10%
- P(Cloud|Rain) is Probability of Cloud, given that Rain happens = 50%
- P(Cloud) is Probability of Cloud = 40%

$P(Rain|Cloud) =  \frac{(0.1 x 0.5)} {0.4}   = .125$

Or a 12.5% chance of rain. Not too bad, let's have a picnic!

**Naïve:** It is called Naïve because it assumes that the occurrence of a certain feature is independent of the occurrence of other features. Such as if the fruit is identified on the bases of color, shape, and taste, then red, spherical, and sweet fruit is recognized as an apple. Hence each feature individually contributes to identify that it is an apple without depending on each other.<br>
**Bayes:** It is called Bayes because it depends on the principle of Bayes' Theorem

# Problem statement

Spam filtering using naive Bayes classifiers in order to predict whether a new mail based on its content, can be categorized as spam or not-spam.

### Data processing using panda library

In [168]:
# Import the required libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt

In [169]:
# Load the dataset

data = pd.read_csv("spam.tsv",sep='\t',names=['Class','Message'])
data.head(8) # View the first 8 records of our dataset

,Class,Message
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!
5,ham,As per your request 'Melle Melle (Oru Minnamin...
6,spam,WINNER!! As a valued network customer you have...
7,spam,Had your mobile 11 months or more? U R entitle...


In [170]:
# to view the first record
data.loc[:0]

,Class,Message
0,ham,I've been searching for the right words to tha...


In [171]:
# Summary of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5567 entries, 0 to 5566
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5567 non-null   object
 1   Message  5567 non-null   object
dtypes: object(2)
memory usage: 87.1+ KB


In [172]:
# create a column to keep the count of the characters present in each record
data['Length'] = data['Message'].apply(len)

In [173]:
data['Length']

0       196
1       155
2        61
3        77
4        36
       ... 
5562    160
5563     36
5564     57
5565    125
5566     26
Name: Length, Length: 5567, dtype: int64

In [174]:
# view the dataset with the column 'Length' which contains the number of characters present in each mail
data.head(10)

,Class,Message,Length
0,ham,I've been searching for the right words to tha...,196
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,ham,"Nah I don't think he goes to usf, he lives aro...",61
3,ham,Even my brother is not like to speak with me. ...,77
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!!,36
5,ham,As per your request 'Melle Melle (Oru Minnamin...,160
6,spam,WINNER!! As a valued network customer you have...,157
7,spam,Had your mobile 11 months or more? U R entitle...,154
8,ham,I'm gonna be home soon and i don't want to tal...,109
9,spam,"SIX chances to win CASH! From 100 to 20,000 po...",136


In [175]:
## The mails are categorised into 2 classes ie., spam and ham.
# Let's see the count of each class
data.groupby('Class').count()

,Message,Length
Class,,
ham,4821,4821
spam,746,746


### Data Visualization

In [176]:
data['Length'].describe() # to find the max length of the message.

count    5567.000000
mean       80.450153
std        59.891023
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: Length, dtype: float64

See what we found, A 910 character long message. Let's use masking to find this message:

In [177]:
data['Length']==910

0       False
1       False
2       False
3       False
4       False
        ...  
5562    False
5563    False
5564    False
5565    False
5566    False
Name: Length, Length: 5567, dtype: bool

In [178]:
# the message that has the max characters
data[data['Length']==910]['Message']

1080    For me the love should start with attraction.i...
Name: Message, dtype: object

In [179]:
# view the message that has 910 characters in it
data[data['Length']==910]['Message'].iloc[0]

"For me the love should start with attraction.i should feel that I need her every time around me.she should be the first thing which comes in my thoughts.I would start the day and end it with her.she should be there every time I dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.I would cry for her.will give all my happiness and take all her sorrows.I will be ready to fight with anyone for her.I will be in love when I will be doing the craziest things for her.love will be when I don't have to proove anyone that my girl is the most beautiful lady on the whole planet.I will always be singing praises for her.love will be when I start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is with me.I would like to say a lot..will tell later.."

In [180]:
# View the message that has min characters
data[data['Length']==2]['Message'].iloc[0]

'Ok'

In [181]:
data.Class

0        ham
1       spam
2        ham
3        ham
4        ham
        ... 
5562    spam
5563     ham
5564     ham
5565     ham
5566     ham
Name: Class, Length: 5567, dtype: object

### Text Pre-Processing

In [182]:
data['Class']=data['Class'].map({"spam":0,'ham':1})
data.Class

0       1
1       0
2       1
3       1
4       1
       ..
5562    0
5563    1
5564    1
5565    1
5566    1
Name: Class, Length: 5567, dtype: int64

In [183]:
data.head(8)

,Class,Message,Length
0,1,I've been searching for the right words to tha...,196
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155
2,1,"Nah I don't think he goes to usf, he lives aro...",61
3,1,Even my brother is not like to speak with me. ...,77
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36
5,1,As per your request 'Melle Melle (Oru Minnamin...,160
6,0,WINNER!! As a valued network customer you have...,157
7,0,Had your mobile 11 months or more? U R entitle...,154


First removing punctuation. We can just take advantage of Python's built-in string library to get a quick list of all the possible punctuation:

In [184]:
# the default list of punctuations
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [185]:
# Why is it important to remove punctuation?

"This message is spam" == "This message is spam."

False

In [186]:
str1="hi... Hello!!..??"

In [187]:
import string

In [188]:
"".join([i for i in str1 if i not in string.punctuation])

'hi Hello'

In [189]:
def remove(text):
  out=''
  for i in text:
    if i not in string.punctuation:
      out=out+i
  return out

remove(str1)



'hi Hello'

In [190]:
"hello"=="Hello"

False

In [191]:
# Let's remove the punctuation

def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text.lower()

data['text_clean'] = data['Message'].apply(lambda x: remove_punct(x))

data.head()

,Class,Message,Length,text_clean
0,1,I've been searching for the right words to tha...,196,ive been searching for the right words to than...
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,free entry in 2 a wkly comp to win fa cup fina...
2,1,"Nah I don't think he goes to usf, he lives aro...",61,nah i dont think he goes to usf he lives aroun...
3,1,Even my brother is not like to speak with me. ...,77,even my brother is not like to speak with me t...
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,i have a date on sunday with will


__Tokenization__ (process of converting the normal text strings in to a list of tokens(also known as lemmas)).

In [192]:
# original text and cleaned text
data.head(8)

,Class,Message,Length,text_clean
0,1,I've been searching for the right words to tha...,196,ive been searching for the right words to than...
1,0,Free entry in 2 a wkly comp to win FA Cup fina...,155,free entry in 2 a wkly comp to win fa cup fina...
2,1,"Nah I don't think he goes to usf, he lives aro...",61,nah i dont think he goes to usf he lives aroun...
3,1,Even my brother is not like to speak with me. ...,77,even my brother is not like to speak with me t...
4,1,I HAVE A DATE ON SUNDAY WITH WILL!!!,36,i have a date on sunday with will
5,1,As per your request 'Melle Melle (Oru Minnamin...,160,as per your request melle melle oru minnaminun...
6,0,WINNER!! As a valued network customer you have...,157,winner as a valued network customer you have b...
7,0,Had your mobile 11 months or more? U R entitle...,154,had your mobile 11 months or more u r entitled...


Now we need to convert each of those messages into a vector the SciKit Learn's algorithm models can work with and machine learning model which we will gonig to use can understand.

In [193]:
# Countvectorizer is a method to convert text to numerical data.

# Initialize the object for countvectorizer
CV = CountVectorizer(stop_words="english")

[Stopwords are the words in any language which does not add much meaning to a sentence. They are the words which are very common in text documents such as a, an, the, you, your, etc. The Stop Words highly appear in text documents. However, they are not being helpful for text analysis in many of the cases, So it is better to remove from the text. We can focus on the important words if stop words have removed.]

In [194]:
CV.fit(data['text_clean'])

CountVectorizer(stop_words='english')

In [195]:
x=CV.transform(data['text_clean']).toarray()

In [196]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [197]:
y=data['Class'].values

In [198]:
y

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [199]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Splitting Train and Test Data

In [200]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [201]:
len(x_train),len(x_test)

(4453, 1114)

### Training a model

With messages represented as vectors, we can finally train our spam/ham classifier. Now we can actually use almost any sort of classification algorithms. For a variety of reasons, the Naive Bayes classifier algorithm is a good choice.

In [202]:
# Initialising the model
NB = MultinomialNB()

In [203]:
NB.fit(x_train,y_train)

MultinomialNB()

In [204]:
y_pred=NB.predict(x_test)

In [205]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [206]:
# Checking accuracy

accuracyScore = accuracy_score(y_test,y_pred)*100

print("Prediction Accuracy :",accuracyScore)

Prediction Accuracy : 96.94793536804309


### SpamClassificationApplication

In [207]:
msg = input("Enter Message: ") # to get the input message
msgInput = CV.transform([msg]) #
predict = NB.predict(msgInput)
if(predict[0]==0):
    print("------------------------MESSAGE-SENT-[CHECK-SPAM-FOLDER]---------------------------")
else:
    print("---------------------------MESSAGE-SENT-[CHECK-INBOX]------------------------------")

Enter Message:  hi ok


---------------------------MESSAGE-SENT-[CHECK-INBOX]------------------------------


## BAG OF WORDS

We cannot pass text directly to train our models in Natural Language Processing, thus we need to convert it into numbers, which machine can understand and can perform the required modelling on it

In [208]:
# Let's understand it with a simple example

In [209]:
# creating a list of sentences
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."]

# Changing the text to lower case and remove the full stop from text
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs[3]

'man eats food'

In [210]:
# corpus is the collection of text
#look at the documents list
print("Our corpus: ", processed_docs)


# Initialise the object for CountVectorizer
count_vect = CountVectorizer()

#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Our corpus:  ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']
Our vocabulary:  {'dog': 1, 'bites': 0, 'man': 4, 'eats': 2, 'meat': 5, 'food': 3}
BoW representation for 'dog bites man':  [[1 1 0 0 1 0]]
BoW representation for 'man bites dog:  [[1 1 0 0 1 0]]
Bow representation for 'dog and dog are friends': [[0 2 0 0 0 0]]
